<h1>This notebook has been created as a part of the Coursera Capstone Project 
for the IBM Data Science Professional Certificate </h1>

In [1]:
import pandas as pd
import numpy as np

print("Hello Capstone Project Course")

Hello Capstone Project Course


<h3>Purpose of this notebook : explore and cluster the neighborhoods in Toronto.</h3>

<h4>Step 1 : Scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, </h4>
<h4>Output : Create a dataframe - </h4>
<ol>
    <li>The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood</li>
    <li>Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.</li>
    <li>More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.</li>
    <li>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.</li>
    <li>Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.</li>
    <li>In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.</li>
</ol>    
    

1.1  Using pandas to read the table from the web page

In [2]:
import pandas as pd
list_of_dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
len(list_of_dfs)

3

In [4]:
list_of_dfs[0]

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [5]:
#since the very first of the dataframes lifted by pandas is the one we need
#we store this separately in a dataframe named unclean_df since we still have plenty
#of wrangling and cleaning to do.

unclean_df = list_of_dfs[0]
unclean_df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [6]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned
#Replacing all values of Not assigned in the Boroughs column with NaN so that these rows can be dropped
import numpy as np
unclean_df.loc[unclean_df['Borough']=='Not assigned'] = np.NaN
unclean_df.dropna(axis=0, inplace=True)
unclean_df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


<h6>Note that the data obtained after cleaning up the rows with Not assigned in the Boroughs column, I found that the other cleanup described was not required</h6>

In [7]:
#To demonstrate that there are no duplicates in the list of postal codes
#number of unique values in the postal code column is 103 which is same as after removing the NaN rows
unclean_df.nunique()

Postal Code      103
Borough           10
Neighbourhood     99
dtype: int64

In [8]:
#to demonstrate that there are no neighbourhoods which contain the value "Not assigned"
notass = unclean_df['Neighbourhood']
notass_df = notass.str.contains("Not assigned", regex=False).to_frame()
notass_df.loc[notass_df['Neighbourhood']==True]

,Neighbourhood


In [9]:
#Shape of the the now clean dataframe
unclean_df.shape

(103, 3)

<h3>-- Step 1 or Question 1 is now COMPLETE --</h3>